In [9]:
import opendatasets as od

In [1]:
dataset = 'https://www.kaggle.com/datasets/rajatkumar30/fake-news'

In [11]:
od.download(dataset,force='True')

  0%|                                                                                      | 0.00/11.5M [00:00<?, ?B/s]

100%|██████████████████████████████████████████████████████████████████████████████| 11.5M/11.5M [01:06<00:00, 182kB/s]


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('news.csv')
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [4]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import warnings
%matplotlib inline

warnings.filterwarnings('ignore')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6335 entries, 0 to 6334
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  6335 non-null   int64 
 1   title       6335 non-null   object
 2   text        6335 non-null   object
 3   label       6335 non-null   object
dtypes: int64(1), object(3)
memory usage: 198.1+ KB


In [6]:
print(df.columns)

Index(['Unnamed: 0', 'title', 'text', 'label'], dtype='object')


In [6]:
df = df.drop(columns=['Unnamed: 0', 'title'], axis=1)
df = df.dropna(axis=0)

In [7]:
len(df)

6335

In [8]:
df['clean_news']=df['text'].str.lower()
df['clean_news']

0       daniel greenfield, a shillman journalism fello...
1       google pinterest digg linkedin reddit stumbleu...
2       u.s. secretary of state john f. kerry said mon...
3       — kaydee king (@kaydeeking) november 9, 2016 t...
4       it's primary day in new york and front-runners...
                              ...                        
6330    the state department told the republican natio...
6331    the ‘p’ in pbs should stand for ‘plutocratic’ ...
6332     anti-trump protesters are tools of the oligar...
6333    addis ababa, ethiopia —president obama convene...
6334    jeb bush is suddenly attacking trump. here's w...
Name: clean_news, Length: 6335, dtype: object

In [9]:
df['clean_news'] = df['clean_news'].str.replace('[^A-Za-z0-9\s]', '')
df['clean_news'] = df['clean_news'].str.replace('\n', '')
df['clean_news'] = df['clean_news'].str.replace('\s+', ' ')
df['clean_news']

0       daniel greenfield a shillman journalism fellow...
1       google pinterest digg linkedin reddit stumbleu...
2       us secretary of state john f kerry said monday...
3        kaydee king kaydeeking november 9 2016 the le...
4       its primary day in new york and frontrunners h...
                              ...                        
6330    the state department told the republican natio...
6331    the p in pbs should stand for plutocratic or p...
6332     antitrump protesters are tools of the oligarc...
6333    addis ababa ethiopia president obama convened ...
6334    jeb bush is suddenly attacking trump heres why...
Name: clean_news, Length: 6335, dtype: object

In [10]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
df['clean_news'] = df['clean_news'].apply(lambda x: " ".join([word for word in x.split() if word not in stop]))
df.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gayat\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,text,label,clean_news
0,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE,daniel greenfield shillman journalism fellow f...
1,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE,google pinterest digg linkedin reddit stumbleu...
2,U.S. Secretary of State John F. Kerry said Mon...,REAL,us secretary state john f kerry said monday st...
3,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE,kaydee king kaydeeking november 9 2016 lesson ...
4,It's primary day in New York and front-runners...,REAL,primary day new york frontrunners hillary clin...


In [11]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gayat\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import multiprocessing


df['tokenized_news'] = df['clean_news'].apply(word_tokenize)


cores = multiprocessing.cpu_count()
w2v_model = Word2Vec(sentences=df['tokenized_news'], 
                     vector_size=100,  
                     window=5,         
                     min_count=1,      
                     workers=cores,    
                     sg=0)            

# Save the trained model
w2v_model.save(r"C:\Users\gayat\OneDrive\Desktop\gen\word2vec_model")

# To load the model later, you can use:
# w2v_model = Word2Vec.load("word2vec_model")


In [14]:
from sklearn.model_selection import train_test_split


X = df['clean_news']
y = df['label'] 


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


print("Training set shape:", X_train.shape, y_train.shape)
print("Testing set shape:", X_test.shape, y_test.shape)


Training set shape: (5068,) (5068,)
Testing set shape: (1267,) (1267,)


In [15]:
from sklearn.base import TransformerMixin
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

cores = multiprocessing.cpu_count()
w2v_model = Word2Vec(sentences=X_train, vector_size=100, window=5, min_count=1, workers=cores, sg=0)

class WordEmbeddingsTransformer(TransformerMixin):
    def __init__(self, word2vec_model):
        self.word2vec_model = word2vec_model

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return np.array([self.get_sentence_embedding(sentence) for sentence in X])

    def get_sentence_embedding(self, sentence):
        
        embeddings = [self.word2vec_model.wv[word] for word in sentence.split() if word in self.word2vec_model.wv]
        return np.mean(embeddings, axis=0) if embeddings else np.zeros(self.word2vec_model.vector_size)


model = make_pipeline(WordEmbeddingsTransformer(w2v_model), LogisticRegression())


model.fit(X_train, y_train)


y_pred = model.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")


print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.59
Classification Report:
               precision    recall  f1-score   support

        FAKE       0.61      0.44      0.51       628
        REAL       0.57      0.73      0.64       639

    accuracy                           0.59      1267
   macro avg       0.59      0.58      0.58      1267
weighted avg       0.59      0.59      0.58      1267



In [16]:
news = ['dog sat on a cow']
model.predict(news)

array(['FAKE'], dtype=object)